# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.ops.gen_array_ops import shape
import random
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Model

In [ ]:
def Def_model(X_test):
  input = tf.keras.Input(shape=X_test[0].shape)
  # CNN
  x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(input)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
  x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
  x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
  x = tf.keras.layers.Flatten()(x)

  output = tf.keras.layers.Dense(64)(x)
  output = tf.keras.layers.Dense(2)(output)
  # output = tf.keras.layers.Dropout(.5,input_shape=(5,))(output)
  output = tf.keras.activations.sigmoid(output)
  global_model = tf.keras.Model(inputs=[input], outputs=[output])
  return global_model

# Hàm đánh giá

In [ ]:
def Evaluation_Function(global_model,X_test,y_test):
    y_predict = global_model.predict(X_test).argmax(1)
    y_true = np.array(y_test).argmax(1)
    acc = tf.keras.metrics.Accuracy()
    acc.update_state(y_true, y_predict)
    acc = acc.result().numpy()
    recall = tf.keras.metrics.Recall()
    recall.update_state(y_true, y_predict)
    recall = recall.result().numpy()
    precision = tf.keras.metrics.Precision()
    precision.update_state(y_true, y_predict)
    precision = precision.result().numpy()
    f_score =  2 * recall * precision / (recall + precision)
    return [acc,recall,precision,f_score]

# Load data

In [ ]:
def load_data(path, test = True,r = 1, agents = 1):
    if test:
        X = pd.read_csv(path)
    else:
        path = path + "PC" + str(agents) + ".csv"
        X = pd.read_csv(path)

    Y = X.iloc[: , -1]
    X= X.iloc[: , :-1]
    num_records = X.shape[0]
    X = tf.constant(X.values[:,0:187])
    X = tf.reshape(X, (num_records, 187, 1))
    Y= tf.one_hot(Y.values, 2)
    Y = tf.reshape(Y, (num_records, 2))
    return X, Y

    #   Y = X.iloc[:,-1:]
    # num_records = Y.shape[0]
    # X = tf.constant(X.values[:,0:30])
    # X = tf.reshape(X, (num_records, 30, 1))
    # Y = tf.one_hot(Y.values[:,0:], 2)
    # Y = tf.reshape(Y, (num_records, 2))
    # return X,Y

In [ ]:
path = "/content/drive/MyDrive/SwarmLearn/Cardio/Dataset/Test.csv"
X_test, y_test = load_data(path)

In [ ]:
from keras.optimizers import SGD
from sklearn.metrics import precision_score, recall_score, f1_score
global_model = Def_model(X_test)
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
global_model.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=["accuracy"]
)

# Dữ liệu bị lật

## FL Model

In [ ]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    st = time.time()
    list_agents_model = []
    global_weights = global_model.get_weights()
    avg_weights = []
    model_save = global_model.get_weights()
    for i in global_weights:
      avg_weights.append(np.zeros_like(i))
    # encryptweightlayer(global_weights)
    sum_data_train = 0
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(tf.keras.models.clone_model(global_model))
        list_agents_model[i].set_weights(global_weights)

    # ratio data
    for i in range(agents):
      path = "/content/drive/MyDrive/SwarmLearn/Cardio/Flip_data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      sum_data_train += len(Y_train)
    # fed avg
    for i in range(agents):
      path = "/content/drive/MyDrive/SwarmLearn/Cardio/Flip_data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 65, epochs=1) #

      local_weights = list_agents_model[i].get_weights()

      ratio = len(Y_train) / sum_data_train

      for j in range(len(local_weights)):
        avg_weights[j] += ratio * (local_weights[j] - model_save[j])

    global_model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=["accuracy"]
    )
    for i in range(len(global_weights)):
      avg_weights[i] = model_save[i] + avg_weights[i]
    global_model.set_weights(avg_weights)

  print('Final Evaluating:')
  [acc,recall,precision,f_score] = Evaluation_Function(global_model, X_test, y_test)
  return acc,recall,precision,f_score

In [ ]:
FL_Train(rounds=5,agents=3,global_model=global_model)

Round  1
Local 0 Training:
452/452 [==============================] - 14s 27ms/step - loss: 0.1425 - accuracy: 0.9533
Local 1 Training:
452/452 [==============================] - 17s 29ms/step - loss: 0.1421 - accuracy: 0.9543
Local 2 Training:
452/452 [==============================] - 15s 27ms/step - loss: 0.1412 - accuracy: 0.9530
Round  2
Local 0 Training:
452/452 [==============================] - 16s 29ms/step - loss: 0.1158 - accuracy: 0.9624
Local 1 Training:
452/452 [==============================] - 14s 24ms/step - loss: 0.1152 - accuracy: 0.9625
Local 2 Training:
452/452 [==============================] - 14s 27ms/step - loss: 0.1120 - accuracy: 0.9646
Round  3
Local 0 Training:
452/452 [==============================] - 16s 30ms/step - loss: 0.1016 - accuracy: 0.9681
Local 1 Training:
452/452 [==============================] - 14s 27ms/step - loss: 0.1017 - accuracy: 0.9678
Local 2 Training:
452/452 [==============================] - 14s 27ms/step - loss: 0.1009 - accuracy:

(0.9718718, 0.84721947, 0.9877711, 0.9121125141155282)

# Dữ liệu bình thường

In [ ]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    st = time.time()
    list_agents_model = []
    global_weights = global_model.get_weights()
    avg_weights = []
    model_save = global_model.get_weights()
    for i in global_weights:
      avg_weights.append(np.zeros_like(i))
    # encryptweightlayer(global_weights)
    sum_data_train = 0
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(tf.keras.models.clone_model(global_model))
        list_agents_model[i].set_weights(global_weights)

    # ratio data
    for i in range(agents):
      path = "/content/drive/MyDrive/SwarmLearn/Cardio/Dataset/"
      X_train, Y_train = load_data(path,False, r, i+1)
      sum_data_train += len(Y_train)
    # fed avg
    for i in range(agents):
      path = "/content/drive/MyDrive/SwarmLearn/Cardio/Dataset/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 65, epochs=1) #

      local_weights = list_agents_model[i].get_weights()

      ratio = len(Y_train) / sum_data_train

      for j in range(len(local_weights)):
        avg_weights[j] += ratio * (local_weights[j] - model_save[j])

    global_model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=["accuracy"]
    )
    for i in range(len(global_weights)):
      avg_weights[i] = model_save[i] + avg_weights[i]
    global_model.set_weights(avg_weights)

  print('Final Evaluating:')
  [acc,recall,precision,f_score] = Evaluation_Function(global_model, X_test, y_test)
  return acc,recall,precision,f_score

In [ ]:
FL_Train(rounds=5,agents=3,global_model=global_model)

Round  1
Local 0 Training:
452/452 [==============================] - 10s 20ms/step - loss: 0.0471 - accuracy: 0.9821
Local 1 Training:
452/452 [==============================] - 9s 19ms/step - loss: 0.0555 - accuracy: 0.9795
Local 2 Training:
452/452 [==============================] - 9s 19ms/step - loss: 0.0474 - accuracy: 0.9828
Round  2
Local 0 Training:
452/452 [==============================] - 9s 18ms/step - loss: 0.0465 - accuracy: 0.9825
Local 1 Training:
452/452 [==============================] - 9s 19ms/step - loss: 0.0541 - accuracy: 0.9803
Local 2 Training:
452/452 [==============================] - 8s 16ms/step - loss: 0.0471 - accuracy: 0.9826
Round  3
Local 0 Training:
452/452 [==============================] - 8s 16ms/step - loss: 0.0461 - accuracy: 0.9823
Local 1 Training:
452/452 [==============================] - 8s 16ms/step - loss: 0.0532 - accuracy: 0.9802
Local 2 Training:
452/452 [==============================] - 8s 16ms/step - loss: 0.0468 - accuracy: 0.9824


(0.9824412, 0.9117356, 0.9852438, 0.9470654992364187)